## Codificación por número de observaciones o frecuencia 


En la primera codificación reemplazamos las categorías por el número de observaciones por categoría en los datos. Similarmente, podemos reemplazar la categoría por la frecuencia -o porcentaje- de observaciones en los datos. Eso es, si 10 de nuestras 100 observaciones muestran el color azul, entonces reemplazamos el color azul por 10 o por 0.1 si reemplazamos por la frecuencia. Estas técnicas capturan la representación de cada etiqueta en los datos, pero la codificación puede que no necesariamente tenga poder predictivo en el target. Sin embargo, estos métodos son bastante populares en las competiciones de Kaggle.

El supuesto de esta técnica es que el número de observaciones presentes en cada una de las categorías de una variable es de alguna forma representativo del poder predictivo de dicha etiqueta.

### Ventajas

- Simple
- No extiende el espacio de los datos (número de variables)

### Desventajas

- Si dos categorías aparecen el mismo número de veces u observaciones en los datos, serán reemplazadas por el mismo númer; la consecuencia es que puede que perdamos información importante.

Por ejemplo, si hay 10 observaciones para la categoría azul y 10 observaciones para la categoría roja, ambas serán reemplazadas por 10, y por lo tanto, luego de codificarlas, parecerán ser la misma cosa


Sigue esta conversación en [en Kaggle](https://www.kaggle.com/general/16927) para más información.



## En este demo:

Vamos a realizar codificación one hot con:
- pandas
- Feature-Engine

y las ventajas y limitaciones de cada una de estas implementaciones usando los datos House Prices dataset.



In [1]:
import numpy as np
import pandas as pd

# separar datos
from sklearn.model_selection import train_test_split

# codificar con feature-engine
from feature_engine.categorical_encoders import CountFrequencyCategoricalEncoder

In [2]:
# cargar dataset

data = pd.read_csv(
    '../houseprice.csv',
    usecols=['Neighborhood', 'Exterior1st', 'Exterior2nd', 'SalePrice'])

data.head()

,Neighborhood,Exterior1st,Exterior2nd,SalePrice
0,CollgCr,VinylSd,VinylSd,208500
1,Veenker,MetalSd,MetalSd,181500
2,CollgCr,VinylSd,VinylSd,223500
3,Crawfor,Wd Sdng,Wd Shng,140000
4,NoRidge,VinylSd,VinylSd,250000


In [3]:
# miremos cuantas etiquetas tiene cada variable

for col in data.columns:
    print(col, ': ', len(data[col].unique()), ' etiquetas')

Neighborhood :  25  etiquetas
Exterior1st :  15  etiquetas
Exterior2nd :  16  etiquetas
SalePrice :  663  etiquetas


### Importante sobre codificación

Cuando hacemos el conteo de observaciones para transformar las variables categóricas, es importante calcular el número ( o frecuencia =  número observaciones /  observaciones totales)  usando el set de entrenamiento; y luego usar estos números para codificar las variables en el set de prueba



In [4]:
# separemos en sets de prueba y entrenamiento

X_train, X_test, y_train, y_test = train_test_split(
    data[['Neighborhood', 'Exterior1st', 'Exterior2nd']],  # predictores
    data['SalePrice'],  # target
    test_size=0.3,  # porcentaje observaciones prueba
    random_state=0)  # semilla para asegurar reproducibilidad

X_train.shape, X_test.shape

((1022, 3), (438, 3))

## Codificación por número de observaciones o frecuencia con pandas

In [5]:
# calculemos para cada una de las etiquetas el número de observaciones
# para la variable Neigbourhood

count_map = X_train['Neighborhood'].value_counts().to_dict()

count_map

{'NAmes': 151,
 'CollgCr': 105,
 'OldTown': 73,
 'Edwards': 71,
 'Sawyer': 61,
 'Somerst': 56,
 'Gilbert': 55,
 'NWAmes': 51,
 'NridgHt': 51,
 'SawyerW': 45,
 'BrkSide': 41,
 'Mitchel': 36,
 'Crawfor': 35,
 'NoRidge': 30,
 'Timber': 30,
 'IDOTRR': 24,
 'ClearCr': 24,
 'SWISU': 18,
 'StoneBr': 16,
 'Blmngtn': 12,
 'MeadowV': 12,
 'BrDale': 10,
 'NPkVill': 7,
 'Veenker': 6,
 'Blueste': 2}

El diccionario contiene el número de observaciones por cada categoría de la variable Neighbourhood.

In [6]:
# reemplacemos las etiquetas con el conteo que hicimos

X_train['Neighborhood'] = X_train['Neighborhood'].map(count_map)
X_test['Neighborhood'] = X_test['Neighborhood'].map(count_map)

In [7]:
# exploremos los resultados

X_train['Neighborhood'].head(10)

64      105
682      24
960      41
1384     71
1100     18
416      61
1034     35
853     151
472      71
1011     71
Name: Neighborhood, dtype: int64

In [8]:
# si en lugar del número de observaciones queremos reemplazar por la frecuencia
# solo necesitamos dividir el conteo por el número total de observaciones

frequency_map = (X_train['Exterior1st'].value_counts() / len(X_train) ).to_dict()
frequency_map

{'VinylSd': 0.3561643835616438,
 'HdBoard': 0.149706457925636,
 'Wd Sdng': 0.14481409001956946,
 'MetalSd': 0.1350293542074364,
 'Plywood': 0.08414872798434442,
 'CemntBd': 0.03816046966731898,
 'BrkFace': 0.03424657534246575,
 'WdShing': 0.02054794520547945,
 'Stucco': 0.016634050880626222,
 'AsbShng': 0.014677103718199608,
 'Stone': 0.0019569471624266144,
 'ImStucc': 0.0009784735812133072,
 'BrkComm': 0.0009784735812133072,
 'AsphShn': 0.0009784735812133072,
 'CBlock': 0.0009784735812133072}

In [9]:
# reemplacemos las categorías por las frecuencias

X_train['Exterior1st'] = X_train['Exterior1st'].map(frequency_map)
X_test['Exterior1st'] = X_test['Exterior1st'].map(frequency_map)

Podemos agrupar estos comandos en dos funciones como hiciemos en los notebooks anteriores y  repetir el proceso (con un ciclo) para cada una de las variables categóricas. Si no sabes como hacer eso, revisa los notebooks anteriores.

## Codificación por número de observaciones o frecuencia con Feature-Engine

In [10]:
# separemos en sets de prueba y entrenamiento

X_train, X_test, y_train, y_test = train_test_split(
    data[['Neighborhood', 'Exterior1st', 'Exterior2nd']],  # variables
    data['SalePrice'],  # target
    test_size=0.3,  # porcentaje observaciones prueba
    random_state=0)  # semilla para asegurar reproducibilidad

X_train.shape, X_test.shape

((1022, 3), (438, 3))

In [11]:
count_enc = CountFrequencyCategoricalEncoder(
    encoding_method='count', # para codificar por frecuencia ==> encoding_method='frequency'
    variables=['Neighborhood', 'Exterior1st', 'Exterior2nd'])

count_enc.fit(X_train)

CountFrequencyCategoricalEncoder(encoding_method='count',
                                 variables=['Neighborhood', 'Exterior1st',
                                            'Exterior2nd'])

In [12]:
# en en atributo encoder_dictdel codificador 
# podemos ver el número de observaciones por categoría de cada variable

count_enc.encoder_dict_

{'Neighborhood': {'NAmes': 151,
  'CollgCr': 105,
  'OldTown': 73,
  'Edwards': 71,
  'Sawyer': 61,
  'Somerst': 56,
  'Gilbert': 55,
  'NWAmes': 51,
  'NridgHt': 51,
  'SawyerW': 45,
  'BrkSide': 41,
  'Mitchel': 36,
  'Crawfor': 35,
  'NoRidge': 30,
  'Timber': 30,
  'IDOTRR': 24,
  'ClearCr': 24,
  'SWISU': 18,
  'StoneBr': 16,
  'Blmngtn': 12,
  'MeadowV': 12,
  'BrDale': 10,
  'NPkVill': 7,
  'Veenker': 6,
  'Blueste': 2},
 'Exterior1st': {'VinylSd': 364,
  'HdBoard': 153,
  'Wd Sdng': 148,
  'MetalSd': 138,
  'Plywood': 86,
  'CemntBd': 39,
  'BrkFace': 35,
  'WdShing': 21,
  'Stucco': 17,
  'AsbShng': 15,
  'Stone': 2,
  'ImStucc': 1,
  'BrkComm': 1,
  'AsphShn': 1,
  'CBlock': 1},
 'Exterior2nd': {'VinylSd': 353,
  'Wd Sdng': 142,
  'HdBoard': 141,
  'MetalSd': 136,
  'Plywood': 112,
  'CmentBd': 39,
  'Wd Shng': 29,
  'BrkFace': 18,
  'AsbShng': 17,
  'Stucco': 16,
  'ImStucc': 8,
  'Stone': 4,
  'Brk Cmn': 4,
  'Other': 1,
  'AsphShn': 1,
  'CBlock': 1}}

In [13]:
X_train = count_enc.transform(X_train)
X_test = count_enc.transform(X_test)

# exploremos el resultado
X_train.head()

,Neighborhood,Exterior1st,Exterior2nd
64,105,364,353
682,24,148,142
960,41,148,112
1384,71,21,29
1100,18,148,142


**Nota**

Si el argumento 'variables' se fija en 'None' (ninguno). entonces el codificador automáticamente identificará  **todas las variables categóricas**. Maravilloso verdad?

El codificador no codificará las variables numéricas. Entonces si algunas de tus variables categóricas son de hecho numéricas, necesitas hacer el 're-cast' o cambio a tipo 'object' antes de usar el codificador.

Si hay una variable en el set de prueba, para el cual el codificador no tiene un número para asignar ( la categoría no estaba presente en el set de entrenamiento), el codificador devolverá un error.

